In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import os

In [2]:
# Paths to datasets
train_dir = r"F:\KULIAH\SEMESTER 7\VISI KOMPUTER\TUBES\card_dataset\train"
test_dir = r"F:\KULIAH\SEMESTER 7\VISI KOMPUTER\TUBES\card_dataset\test"
val_dir = r"F:\KULIAH\SEMESTER 7\VISI KOMPUTER\TUBES\card_dataset\valid"

In [3]:
# Image size and parameters
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

In [4]:
# Data Augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Simple preprocessing for validation and test
val_test_datagen = ImageDataGenerator(rescale=1.0/255)

# Load training, validation, and test data
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

val_data = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

test_data = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False  # Keep test order consistent for evaluation
)

Found 7624 images belonging to 53 classes.
Found 265 images belonging to 53 classes.
Found 265 images belonging to 53 classes.


In [9]:
# Load MobileNetV2 as base model
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base model layers

# Build the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.3),  # Dropout to prevent overfitting
    Dense(53, activation="softmax")  # 52 classes for cards
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

In [11]:
# Train the model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=50
)

Epoch 1/50
239/239 ━━━━━━━━━━━━━━━━━━━━ 126s 519ms/step - accuracy: 0.4597 - loss: 1.8720 - val_accuracy: 0.4377 - val_loss: 1.8349
Epoch 2/50
239/239 ━━━━━━━━━━━━━━━━━━━━ 126s 522ms/step - accuracy: 0.4546 - loss: 1.8642 - val_accuracy: 0.4302 - val_loss: 1.8212
Epoch 3/50
239/239 ━━━━━━━━━━━━━━━━━━━━ 130s 539ms/step - accuracy: 0.4728 - loss: 1.8071 - val_accuracy: 0.4340 - val_loss: 1.8095
Epoch 4/50
239/239 ━━━━━━━━━━━━━━━━━━━━ 127s 526ms/step - accuracy: 0.4679 - loss: 1.8166 - val_accuracy: 0.4302 - val_loss: 1.7987
Epoch 5/50
239/239 ━━━━━━━━━━━━━━━━━━━━ 128s 530ms/step - accuracy: 0.4773 - loss: 1.8028 - val_accuracy: 0.4415 - val_loss: 1.7962
Epoch 6/50
239/239 ━━━━━━━━━━━━━━━━━━━━ 127s 524ms/step - accuracy: 0.4865 - loss: 1.7902 - val_accuracy: 0.4415 - val_loss: 1.7902
Epoch 7/50
239/239 ━━━━━━━━━━━━━━━━━━━━ 132s 545ms/step - accuracy: 0.4883 - loss: 1.7550 - val_accuracy: 0.4528 - val_loss: 1.7892
Epoch 8/50
239/239 ━━━━━━━━━━━━━━━━━━━━ 127s 524ms/step - accuracy: 0.4872 -

In [12]:
# Fine-tune the model
base_model.trainable = True  # Unfreeze some layers of base model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

In [13]:
# Continue training with fine-tuning
history_fine = model.fit(
    train_data,
    validation_data=val_data,
    epochs=50
)

Epoch 1/50
239/239 ━━━━━━━━━━━━━━━━━━━━ 537s 2s/step - accuracy: 0.1991 - loss: 3.1357 - val_accuracy: 0.4679 - val_loss: 1.6733
Epoch 2/50
239/239 ━━━━━━━━━━━━━━━━━━━━ 516s 2s/step - accuracy: 0.3210 - loss: 2.3230 - val_accuracy: 0.4830 - val_loss: 1.5902
Epoch 3/50
239/239 ━━━━━━━━━━━━━━━━━━━━ 513s 2s/step - accuracy: 0.3781 - loss: 2.0824 - val_accuracy: 0.5208 - val_loss: 1.5097
Epoch 4/50
239/239 ━━━━━━━━━━━━━━━━━━━━ 497s 2s/step - accuracy: 0.4501 - loss: 1.8543 - val_accuracy: 0.5321 - val_loss: 1.4391
Epoch 5/50
239/239 ━━━━━━━━━━━━━━━━━━━━ 493s 2s/step - accuracy: 0.4758 - loss: 1.7168 - val_accuracy: 0.5698 - val_loss: 1.3584
Epoch 6/50
239/239 ━━━━━━━━━━━━━━━━━━━━ 492s 2s/step - accuracy: 0.5098 - loss: 1.6295 - val_accuracy: 0.5811 - val_loss: 1.2993
Epoch 7/50
239/239 ━━━━━━━━━━━━━━━━━━━━ 491s 2s/step - accuracy: 0.5367 - loss: 1.5445 - val_accuracy: 0.6038 - val_loss: 1.2313
Epoch 8/50
239/239 ━━━━━━━━━━━━━━━━━━━━ 493s 2s/step - accuracy: 0.5671 - loss: 1.4360 - val_accu

In [14]:
# Save the improved model
model.save("improved_card_model.h5")

In [15]:
# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_data)
print(f"\nTest Accuracy: {test_accuracy * 100:.2f}%")

9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 529ms/step - accuracy: 0.8666 - loss: 0.3649

Test Accuracy: 84.15%


In [16]:
# Generate predictions for test data
y_pred = model.predict(test_data)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_data.classes

# Print classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=test_data.class_indices.keys()))

9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 439ms/step

Classification Report:
                   precision    recall  f1-score   support

     ace of clubs       0.83      1.00      0.91         5
  ace of diamonds       1.00      1.00      1.00         5
    ace of hearts       1.00      1.00      1.00         5
    ace of spades       1.00      1.00      1.00         5
   eight of clubs       0.80      0.80      0.80         5
eight of diamonds       0.56      1.00      0.71         5
  eight of hearts       1.00      1.00      1.00         5
  eight of spades       0.83      1.00      0.91         5
    five of clubs       0.75      0.60      0.67         5
 five of diamonds       1.00      0.80      0.89         5
   five of hearts       0.83      1.00      0.91         5
   five of spades       1.00      0.80      0.89         5
    four of clubs       1.00      0.60      0.75         5
 four of diamonds       0.83      1.00      0.91         5
   four of hearts       1.00      0.80      0.89   

In [17]:
# Print confusion matrix
print("\nConfusion Matrix:")
conf_matrix = confusion_matrix(y_true, y_pred_classes)
print(conf_matrix)


Confusion Matrix:
[[5 0 0 ... 0 0 0]
 [0 5 0 ... 0 0 0]
 [0 0 5 ... 0 0 0]
 ...
 [0 0 0 ... 4 0 0]
 [0 0 0 ... 1 4 0]
 [0 0 0 ... 0 0 3]]
